In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Read MIMICs CSV file
mimic_df = pd.read_csv("CSV\\exports\\final\\mimic_mean_final.csv")

# Read eICUs CSV file
eicu_df = pd.read_csv("CSV\\exports\\final\\eicu_mean_final.csv")

# Define Categorical - Numerical Data

In [3]:
# Define exclude columns
exclude_columns = ['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count', 'Time_Zone']

# Separating categorical and numerical columns for MIMIC
mimic_categorical_columns = mimic_df.select_dtypes(include=['object', 'category']).columns.tolist()
mimic_numerical_columns = mimic_df.select_dtypes(include=['number']).columns.tolist()

for col in exclude_columns:
    if col in mimic_numerical_columns:
        mimic_numerical_columns.remove(col)
    if col in mimic_categorical_columns:
        mimic_categorical_columns.remove(col)

# Separating categorical and numerical columns for eICU
eicu_categorical_columns = eicu_df.select_dtypes(include=['object', 'category']).columns.tolist()
eicu_numerical_columns = eicu_df.select_dtypes(include=['number']).columns.tolist()

for col in exclude_columns:
    if col in eicu_numerical_columns:
        eicu_numerical_columns.remove(col)
    if col in eicu_categorical_columns:
        eicu_categorical_columns.remove(col)

# Separate Training - Validate - Test - External

In [4]:
# Group by `subject_id` and `hadm_id` to get unique patient admission records
unique_patients = mimic_df[['subject_id', 'hadm_id']].drop_duplicates()

# Split the unique patients into train, validation, and test sets
train_patients, test_patients = train_test_split(unique_patients, test_size=0.10, random_state=42)
train_patients, validate_patients = train_test_split(train_patients, test_size=0.11, random_state=42)  # 0.11 * 90% ~= 10%

# Merge the patients back with the original data to get the full records
train_set = mimic_df.merge(train_patients, on=['subject_id', 'hadm_id'])
validate_set = mimic_df.merge(validate_patients, on=['subject_id', 'hadm_id'])
test_set = mimic_df.merge(test_patients, on=['subject_id', 'hadm_id'])

# External validation from eICU
X_external = eicu_df.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count'])
y_external = eicu_df['hospital_expire_flag']

# Separate features and target for the training, validation, and test sets
X_train = train_set.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count'])
y_train = train_set['hospital_expire_flag']

X_validate = validate_set.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count'])
y_validate = validate_set['hospital_expire_flag']

X_test = test_set.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count'])
y_test = test_set['hospital_expire_flag']

# Preprocess

In [5]:
# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        #('num', StandardScaler(), mimic_numerical_columns),   # Standardize numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), mimic_categorical_columns)   # One-hot encode categorical features
    ])

# Fit and transform the training set
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Apply the same transformations to validation, test, and external sets
X_validate_preprocessed = preprocessor.transform(X_validate)
X_test_preprocessed = preprocessor.transform(X_test)
X_external_preprocessed = preprocessor.transform(X_external)

# ANN

In [ ]:
# Build the ANN model
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    return model

def compile_ann(model):
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

# Get the number of input features after preprocessing
input_dim = X_train_preprocessed.shape[1]  # This is the number of features post-encoding

# Build and compile the ANN
model = build_ann(input_dim)
model = compile_ann(model)

# Train the model using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(X_train_preprocessed, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_validate_preprocessed, y_validate),
                    callbacks=[early_stopping])

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

# External validation on the eICU dataset
external_loss, external_accuracy = model.evaluate(X_external_preprocessed, y_external)
print(f"External Validation Accuracy: {external_accuracy*100:.2f}%")

In [6]:
# Build the ANN model
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    return model

def compile_ann(model):
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

# Get the number of input features after preprocessing
input_dim = X_train_preprocessed.shape[1]  # This is the number of features post-encoding

# Build and compile the ANN
model = build_ann(input_dim)
model = compile_ann(model)

# Train the model using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_preprocessed, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_validate_preprocessed, y_validate),
                    callbacks=[early_stopping])

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

# Predict on the test set
y_test_prob = model.predict(X_test_preprocessed).ravel()
y_test_pred = (y_test_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate precision, recall, F1 score, and confusion matrix
print("\nTest Set Metrics:")
print(classification_report(y_test, y_test_pred, target_names=['Survive', 'Not Survive']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

# External validation on the eICU dataset
external_loss, external_accuracy = model.evaluate(X_external_preprocessed, y_external)
print(f"External Validation Accuracy: {external_accuracy*100:.2f}%")

# Predict on the external validation set
y_external_prob = model.predict(X_external_preprocessed).ravel()
y_external_pred = (y_external_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate precision, recall, F1 score, and confusion matrix for external validation
print("\nExternal Validation Metrics:")
print(classification_report(y_external, y_external_pred, target_names=['Survive', 'Not Survive']))
print("Confusion Matrix:\n", confusion_matrix(y_external, y_external_pred))


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7965 - loss: 0.5065 - val_accuracy: 0.7890 - val_loss: 0.5045
Epoch 2/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7981 - loss: 0.4832 - val_accuracy: 0.7890 - val_loss: 0.5116
Epoch 3/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7992 - loss: 0.4816 - val_accuracy: 0.7890 - val_loss: 0.5155
Epoch 4/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7990 - loss: 0.4798 - val_accuracy: 0.7890 - val_loss: 0.5174
Epoch 5/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7975 - loss: 0.4818 - val_accuracy: 0.7890 - val_loss: 0.5209
Epoch 6/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8009 - loss: 0.4760 - val_accuracy: 0.7890 - val_loss: 0.5207
Epoch 7/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8001 - loss: 0.4763 - val_accuracy: 0.7890 - val_loss: 0.5286
Epoch 8/100
1396/1396 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7989 - loss: 0

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


2696/2696 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9420 - loss: 0.3141
External Validation Accuracy: 93.43%
2696/2696 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step

External Validation Metrics:
              precision    recall  f1-score   support

     Survive       0.93      1.00      0.97     80608
 Not Survive       0.00      0.00      0.00      5664

    accuracy                           0.93     86272
   macro avg       0.47      0.50      0.48     86272
weighted avg       0.87      0.93      0.90     86272

Confusion Matrix:
 [[80608     0]
 [ 5664     0]]


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
